In [47]:
#import box
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [60]:
#loading in data
BrentOilPrices = pd.read_csv('BrentOilPrices.csv')
#plt.plot(BrentOilPrices['Price'])
BrentOilPrices.std()
BrentOilPricesGlobal = pd.read_csv('DP_LIVE_06042021234723780.csv',delimiter=',')

'''for i in BrentOilPricesGlobal['LOCATION']:
    if i == 'AUS':
        print(BrentOilPricesGlobal[i]['PRICE'])'''

'''BrentOilAus = BrentOilPricesGlobal.iloc[0:42]
BrentOilAusVal = BrentOilAus['Value']
plt.plot(range(len(BrentOilAus['TIME'])),BrentOilAusVal)

BrentOilAut = BrentOilPricesGlobal.iloc[43:81]
BrentOilAutVal = BrentOilAut['Value']
plt.plot(range(len(BrentOilAut['TIME'])),BrentOilAutVal)

#right now our x axis is in undefined units'''

BrentOilPrices

,Date,Price
0,20-May-87,18.63
1,21-May-87,18.45
2,22-May-87,18.55
3,25-May-87,18.60
4,26-May-87,18.63
...,...,...
8549,19-Jan-21,55.38
8550,20-Jan-21,55.66
8551,21-Jan-21,55.68
8552,22-Jan-21,55.22
